In [37]:
import numpy as np
import pickle
import pandas as pd
from IPython.display import display
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import ElasticNet, SGDRegressor, Ridge
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [3]:
df = pd.read_excel("ENB2012_data.xlsx")

In [215]:
%%file load_data.py
import pandas as pd
from sklearn.model_selection import train_test_split
def load_and_split_data(file_path, test_size=0.2, seed=22793):
    df = pd.read_excel(file_path)
    train_cols = list(df.columns)[:-2]
    test_cols = list(df.columns)[-2:]
    X = df[train_cols]
    Y1 = df[test_cols[0]]
    Y2 = df[test_cols[1]]
    X_train, X_test, Y1_train, Y1_test, Y2_train, Y2_test = train_test_split(X, Y1, Y2, test_size=test_size, random_state=seed)
    return X_train, X_test, Y1_train, Y1_test, Y2_train, Y2_test

Overwriting load_data.py


In [15]:
df.describe()

def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

display(calc_vif(df.drop(['Y1', 'Y2', 'X2', 'X1', 'X3', 'X6'], axis=1)))

df.drop(['Y1', 'Y2'], axis=1).corr()
# np.linalg.svd(df.drop(['Y1', 'Y2'], axis=1))

,variables,VIF
0,X4,5.245242
1,X5,4.544944
2,X7,4.284222
3,X8,4.485473


,X1,X2,X3,X4,X5,X6,X7,X8
X1,1.000000e+00,-9.919015e-01,-2.037817e-01,-8.688234e-01,8.277473e-01,0.0,1.283986e-17,1.764620e-17
X2,-9.919015e-01,1.000000e+00,1.955016e-01,8.807195e-01,-8.581477e-01,0.0,1.318356e-16,-3.558613e-16
X3,-2.037817e-01,1.955016e-01,1.000000e+00,-2.923165e-01,2.809757e-01,0.0,-7.969726e-19,0.000000e+00
X4,-8.688234e-01,8.807195e-01,-2.923165e-01,1.000000e+00,-9.725122e-01,0.0,-1.381805e-16,-1.079129e-16
X5,8.277473e-01,-8.581477e-01,2.809757e-01,-9.725122e-01,1.000000e+00,0.0,1.861418e-18,0.000000e+00
X6,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,0.000000e+00,0.000000e+00
X7,1.283986e-17,1.318356e-16,-7.969726e-19,-1.381805e-16,1.861418e-18,0.0,1.000000e+00,2.129642e-01
X8,1.764620e-17,-3.558613e-16,0.000000e+00,-1.079129e-16,0.000000e+00,0.0,2.129642e-01,1.000000e+00


In [12]:
# Check for any nulls in the data
for col in df.columns:
    display(df[df[col].isnull()])

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2


,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2


,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2


,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2


,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2


,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2


,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2


,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2


,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2


,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2


### As a baseline, two linear regression models will be trained independently on each target

In [40]:
seed = 22793

In [41]:
train_cols = list(df.columns)[:-2]
# train_cols = ['X4', 'X5', 'X6', 'X7']
test_cols = list(df.columns)[-2:]
X = df[train_cols]
Y1 = df[test_cols[0]]
Y2 = df[test_cols[1]]
X_train, X_test, Y1_train, Y1_test, Y2_train, Y2_test = train_test_split(X, Y1, Y2, test_size=0.2, random_state=seed)

kfolds = 5
scaler = StandardScaler()
scoring = 'neg_mean_squared_error'
# data is already centered, do not need intercept term
model_1 = make_pipeline(StandardScaler(), SGDRegressor(max_iter=10000))
model_2 = make_pipeline(StandardScaler(), SGDRegressor(max_iter=10000))
param_grid={'sgdregressor__alpha': np.arange(0.0, .015, .002)}
grid_search_1 = GridSearchCV(model_1, param_grid, scoring=scoring)
grid_search_2 = GridSearchCV(model_2, param_grid, scoring=scoring)

grid_search_1.fit(X_train, Y1_train, )
grid_search_2.fit(X_train, Y1_train, )

GridSearchCV(estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('sgdregressor',
                                        SGDRegressor(max_iter=10000))]),
             param_grid={'sgdregressor__alpha': array([0.   , 0.002, 0.004, 0.006, 0.008, 0.01 , 0.012, 0.014])},
             scoring='neg_mean_squared_error')

In [42]:
grid_search_1.cv_results_
print(grid_search_1.best_params_)
print(grid_search_2.best_params_)

{'sgdregressor__alpha': 0.0}
{'sgdregressor__alpha': 0.0}


In [43]:
total_mse = grid_search_1.best_score_ + grid_search_2.best_score_
total_mse = total_mse / 2
total_mse

-9.33031087368259

In [44]:
# Fit best model on entire training set and calculate test error
model_1 = make_pipeline(StandardScaler(), SGDRegressor(alpha=0, max_iter=10000))
model_2 = make_pipeline(StandardScaler(), SGDRegressor(alpha=0, max_iter=10000))
model_1 = model_1.fit(X_train, Y1_train)
model_2 = model_2.fit(X_train, Y2_train)


In [46]:
model_1_mse = mean_squared_error(model_1.predict(X_test), Y1_test)
model_2_mse = mean_squared_error(model_2.predict(X_test), Y2_test)
total_test_mse = np.mean([model_1_mse, model_2_mse])

model_1_mse = mean_squared_error(model_1.predict(X_train), Y1_train)
model_2_mse = mean_squared_error(model_2.predict(X_train), Y2_train)
total_train_mse = np.mean([model_1_mse, model_2_mse])

In [47]:
print(total_test_mse)
print(total_train_mse)

7.569671141625257
9.947043429078988


##### Multitarget Ridge Regression

In [5]:
from load_data import load_and_split_data
data_slices = load_and_split_data("ENB2012_data.xlsx")
X_train = data_slices[0]
X_test = data_slices[1]
Y1_train = data_slices[2]
Y1_test = data_slices[3]
Y2_train = data_slices[4]
Y2_test = data_slices[5]

Y_train = pd.concat([Y1_train, Y2_train], axis=1) 
Y_test = pd.concat([Y1_test, Y2_test], axis=1)

In [13]:
kfolds = 5
scaler = StandardScaler()
scoring = 'neg_mean_squared_error'
model_name = "Ridge".lower()
# data is already centered, do not need intercept term
pipeline = make_pipeline(StandardScaler(), Ridge())

param_grid={
            model_name+'__alpha': np.arange(0.01, .2, .005),
            
}
grid_search = GridSearchCV(pipeline, param_grid, scoring=scoring, cv=kfolds)

grid_search.fit(X_train, Y_train)
print(grid_search.best_score_)
grid_search.best_params_

-10.023311361779092


{'ridge__alpha': 0.09999999999999998}

In [14]:
ridge_model = grid_search.best_estimator_

ridge_train_mse = mean_squared_error(ridge_model.predict(X_train), Y_train)
ridge_test_mse = mean_squared_error(ridge_model.predict(X_test), Y_test)
print(ridge_train_mse)
print(ridge_test_mse)

9.790102537993132
7.586827099116071


#### Ridge dropped columns
Remove correlated colums

In [34]:
drop_list = ['X1',  'X3', 'X6']
display(calc_vif(df.drop(['Y1', 'Y2'] + drop_list, axis=1)))

,variables,VIF
0,X2,240.467724
1,X4,150.855951
2,X5,19.114091
3,X7,4.285088
4,X8,4.486469


In [35]:
X_train_droped = X_train.drop(drop_list, axis=1)
kfolds = 5
scaler = StandardScaler()
scoring = 'neg_mean_squared_error'
model_name = "Ridge".lower()
# data is already centered, do not need intercept term
pipeline = make_pipeline(StandardScaler(), Ridge())

param_grid={
            model_name+'__alpha': np.arange(0.1, 1, .1),
            
}
grid_search = GridSearchCV(pipeline, param_grid, scoring=scoring, cv=kfolds)

grid_search.fit(X_train_droped, Y_train)
print(grid_search.best_score_)
grid_search.best_params_

-10.528678809249561


{'ridge__alpha': 0.4}

In [36]:
X_test_dropped = X_test.drop(drop_list, axis=1)
ridge_model_dropped = grid_search.best_estimator_

ridge_dropped_train_mse = mean_squared_error(ridge_model_dropped.predict(X_train_droped), Y_train)
ridge_dropped_test_mse = mean_squared_error(ridge_model_dropped.predict(X_test_dropped), Y_test)
print(ridge_dropped_train_mse)
print(ridge_dropped_test_mse)

10.339729029274592
7.858945109313897
